# Importing Dependencies

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import os

In [2]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Data Preprocessing

In [3]:
# === Settings ===
image_size = (128, 128)
batch_size = 32

In [4]:
def get_image_filepaths_and_class_labels(data_dir):
    class_names = sorted(os.listdir(data_dir))
    filepaths = []
    class_indices = []
    for class_index, class_name in enumerate(class_names):
        class_folder = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_folder):
            continue
        for filename in os.listdir(class_folder):
            if filename.lower().endswith((".jpg", ".jpeg", ".png")):
                filepaths.append(os.path.join(class_folder, filename))
                class_indices.append(class_index)
    return filepaths, class_indices, class_names

In [5]:
def preprocess_image_cv(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Failed to load image: {image_path}")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, image_size)
    image = image.astype(np.float32) / 255.0
    return image


In [6]:
def to_one_hot(index, total_classes):
    encoded = np.zeros(total_classes, dtype=np.float32)
    encoded[index] = 1.0
    return encoded

In [7]:
def load_image_dataset_cv(data_dir):
    filepaths, class_indices, class_names = get_image_filepaths_and_class_labels(data_dir)
    total_classes = len(class_names)
    print(f"Found {len(filepaths)} images across {total_classes} classes.")
    
    image_data = []
    encoded_labels = []
    for path, label in zip(filepaths, class_indices):
        image = preprocess_image_cv(path)
        one_hot_label = to_one_hot(label, total_classes)
        image_data.append(image)
        encoded_labels.append(one_hot_label)
    return image_data, encoded_labels, class_names

In [8]:
def batch_images_and_labels(image_data, encoded_labels, batch_size):
    all_batches = []
    for start in range(0, len(image_data), batch_size):
        batch_images = np.array(image_data[start:start+batch_size])
        batch_labels = np.array(encoded_labels[start:start+batch_size])
        all_batches.append((batch_images, batch_labels))
    return all_batches

# Load dataset and create batches
image_data, encoded_labels, class_names = load_image_dataset_cv("New Plant Diseases Dataset(Augmented)/train")
batches = batch_images_and_labels(image_data, encoded_labels, batch_size)

# Display information about the first batch
first_batch_imgs, first_batch_lbls = batches[0]
print("First batch shapes:", first_batch_imgs.shape, first_batch_lbls.shape)

In [ ]:
# Load train data
train_images, train_labels, class_names = load_image_dataset_cv("New Plant Diseases Dataset(Augmented)/train")

# Create train batches
train_batches = batch_images_and_labels(train_images, train_labels, batch_size)

# Inspect
print("Train batch shape:", train_batches[0][0].shape, train_batches[0][1].shape)

Found 70295 images across 38 classes.


In [11]:
def batch_generator(train_batches):
    for batch_images, batch_labels in train_batches:
        yield batch_images, batch_labels

def convert_batches_to_tf_dataset(train_batches):
    first_batch = train_batches[0]
    output_signature = (
        tf.TensorSpec(shape=first_batch[0].shape[1:], dtype=tf.float32),
        tf.TensorSpec(shape=first_batch[1].shape[1:], dtype=tf.float32)
    )
    dataset = tf.data.Dataset.from_generator(
        lambda: batch_generator(train_batches),
        output_signature=(tf.TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32),
                          tf.TensorSpec(shape=(None, len(train_batches[0][1][0])), dtype=tf.float32))
    )
    return dataset.prefetch(tf.data.AUTOTUNE)


In [12]:
train_ds = convert_batches_to_tf_dataset(train_batches)

In [13]:
train_ds

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 38), dtype=tf.float32, name=None))>